In [1]:
import sys
sys.path.append("..")  # to import from parent dir

In [6]:
from data import REGISTRY
train_ds, test_ds = REGISTRY["cifar100_imb"](data_dir="./data", mode="even", downsample_frac=1)


In [7]:
train_ds

Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=True)
               RandomCrop(size=(224, 224), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [5]:
from collections import Counter
import numpy as np

def subset_class_counts(subset, num_classes=100):
    full_targets = getattr(subset.dataset, "targets", None) or getattr(subset.dataset, "labels")
    indices = list(subset.indices) if hasattr(subset, "indices") else list(range(len(subset)))
    labels = [full_targets[i] for i in indices]
    cnt = Counter(labels)
    counts = np.zeros(num_classes, dtype=int)
    for cls, c in cnt.items():
        counts[cls] = c
    return counts

# use train_ds instead of train_subset
counts = subset_class_counts(train_ds, num_classes=100)

print("Total samples:", counts.sum())
for cls in range(10):  # print first 10 classes as an example
    print(f"class {cls:02d}: {counts[cls]} samples")


AttributeError: 'CIFAR100' object has no attribute 'dataset'

In [21]:
import sys
from pathlib import Path
proj = Path("/home/user/abin_ref_papers/project_structure_demo/dnn_template")
if str(proj) not in sys.path:
    sys.path.insert(0, str(proj))

# ensure any backbone registration file runs
try:
    import models.backbone
except Exception:
    pass

from models import REGISTRY
print("Registered model keys:", sorted(REGISTRY.keys())[:100])


Registered model keys: ['densenet121', 'efficientnet_v2_s', 'resltresnet32', 'resnet18', 'resnet34', 'resnet50', 'resnet50_cbam', 'resnet_cifar_classifier']


In [2]:

import sys
sys.path.append("..")  # to import from parent dir
import torch
import models.backbone   # ensure registrations executed
from models import REGISTRY

# instantiate DAIIC ResNet34 for CIFAR-100 imbalanced experiments:
net = REGISTRY['daiic_resnet34'](num_classes=100, pretrained=True, cifar_stem=True, fusion='concat')
out = net(torch.randn(4,3,32,32))
print(out['logits'].shape, out['W'].shape, out['F'].shape, out['Fprime'].shape)


torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 512, 4, 4]) torch.Size([4, 512, 4, 4])


In [1]:
import sys
sys.path.append("..")

In [6]:
from models import REGISTRY as MODEL_REG
import models.backbone

In [15]:

# initialise model
model = MODEL_REG["daiic_resnet34"](num_classes=100, pretrained=False, cifar_stem=True, in_channel=3)


In [23]:
import torch
from torch import nn



# 1. Initialise the model
# model = daiic_resnet34(
#     num_classes=100,
#     pretrained=False,     # turn off pretrained first to avoid mismatch
#     in_channels=3,
#     cifar_stem=True
# )

# 2. Create dummy CIFAR-100 input: (batch=4, channels=3, height=32, width=32)
x = torch.randn(4, 3, 32, 32)

# 3. Run forward and check shapes
with torch.no_grad():
    out = model(x)
    print("Output logits:", out.shape)   # should be (4, 100)

# 4. If you also want intermediate shapes:
def debug_forward(m, x, name):
    def hook(_, inp, out):
        print(f"{name}: {out.shape}")
    m.register_forward_hook(hook)

# Example: attach hooks to feature extractor layers
for name, layer in model.feature_extractor.named_children():
    debug_forward(model.feature_extractor._modules[name], x, name)

# Now run again
out = model(x)


AttributeError: 'dict' object has no attribute 'shape'

In [24]:
out.keys()

dict_keys(['logits', 'probs', 'W', 'F', 'Fprime'])

In [29]:
out['Fprime'].shape

torch.Size([4, 512, 4, 4])